# PILLOW
Great video for help: https://www.youtube.com/watch?v=5QR-dG68eNE&t=39s
Explanation on anchor points at 1:54:00. May help if text placement needs to be very exact

In [2]:
#import pillow for image manipulation
from PIL import Image, ImageDraw, ImageFont

from datetime import datetime #for saving the figure using today's date

In [3]:
#figure out your current working directory (cwd)
import os # #for working with directory
cwd = os.getcwd() #store original directory into cwd just incase
print("Current working directory: {0}".format(cwd))

Current working directory: c:\Users\bmilinic\OneDrive - DOI\Documents\Python


In [4]:
#change it
os.chdir(r'C:\Users\bmilinic\OneDrive - DOI\Documents\Python') #make sure directory is at default
os.chdir('../ArcGIS/geogenics_source/figures') #change it relative to default directory
cwd = os.getcwd() 
print("New current working directory", cwd)

New current working directory c:\Users\bmilinic\OneDrive - DOI\Documents\ArcGIS\geogenics_source\figures


# 3 x 2 Processing Figures
12/20/2022

### Set up path and examine directory/image sizes

In [24]:
#change directory to 'process' in a way that maintains original directory
path = cwd + '\process' 
os.chdir(path)
#create an object out of my directory
directory = os.scandir(path) 
#loop thorough that object to inspect amount and size of images
n = 1
for file in directory: # for entry in object
    image = Image.open(file.name)
    print(n, image.filename)
    print(image.size)
    n += 1
    
#create list to be used as 'mapnames' variable
directory = os.scandir(path) #this line needs to be repeated...not exactly sure why
for file in directory:
    print("'"+file.name+"',")

1 map%_AsExceed5_ranked.png
(1056, 816)
2 map%_AsExeed5_calculated.png
(1056, 816)
3 map%_GW_DW_calculated.png
(1056, 816)
4 map%_GW_DW_ranked.png
(1056, 816)
5 mapAbs_GW_change_ranked.png
(1056, 816)
6 mapAbs_PPT_change_ranked.png
(1056, 816)
7 mapActive Mines_raw.png
(1056, 816)
8 mapArsenic Prob gt 5.png
(1056, 816)
9 mapBrackish_ft_raw.png
(1056, 816)
10 mapCONUS_boundary.png
(1056, 816)
11 mapCoWU2015_raw.png
(1056, 816)
12 mapData_Gap_calculated.png
(1056, 816)
13 mapData_Gap_ranked.png
(1056, 816)
14 mapDemographic Index_raw.png
(1056, 816)
15 mapDom_well_pop_calculated.png
(1056, 816)
16 mapDom_well_pop_ranked.png
(1056, 816)
17 mapEPARegulatedSites_calculated.png
(1056, 816)
18 mapEPARegulatedSites_ranked.png
(1056, 816)
19 mapEPARegulatedSites_raw.png
(1056, 816)
20 mapGW Storage change_raw.png
(1056, 816)
21 mapHydrologic Regions.png
(1056, 816)
22 mapIR_WU_calculated.png
(1056, 816)
23 mapIR_WU_ranked.png
(1056, 816)
24 mapMag_GW_change_calculated.png
(1056, 816)
25 mapMag_

### LOOPING THROUGH EACH CANVAS

In [25]:
#setup list of map names lists (NEW ORDER)
#organized by order they show up in the paper in groups of 6 maps (3 across and 2 down)
mapnames = [
    ['mapArsenic Prob gt 5.png','map%_AsExeed5_calculated.png','map%_AsExceed5_ranked.png',
     'mapBrackish_ft_raw.png','mapShallow_brackish_calculated.png', 'mapShallow_brackish_ranked.png', 
    ],
    ['mapGW Storage change_raw.png','mapMag_GW_change_calculated.png','mapAbs_GW_change_ranked.png',
     'mapPPT_change_raw.png','mapMag_PPT_change_calculated.png','mapAbs_PPT_change_ranked.png',     
    ],
    ['mapPublic Supply - GW_raw.png','map%_GW_DW_calculated.png','map%_GW_DW_ranked.png',
     'mapCoWU2015_raw.png','mapIR_WU_calculated.png','mapIR_WU_ranked.png',     
    ],
    ['mapEPARegulatedSites_raw.png', 'mapEPARegulatedSites_calculated.png','mapEPARegulatedSites_ranked.png',
     'mapActive Mines_raw.png', 'mapMines_calculated.png', 'mapMines_ranked.png',    
    ],
    ['mapNo. Ppl - Domestic supply_raw.png','mapDom_well_pop_calculated.png','mapDom_well_pop_ranked.png',
     'mapDemographic Index_raw.png','mapSD_calculated.png','mapSD_ranked.png',
    ],
    ['mapNativePop_raw.png','mapNative_pop_den_calculated.png','mapNative_pop_den_ranked.png',
     'mapNEW_TraceALL_Final_Point_raw.png','mapData_Gap_calculated.png', 'mapData_Gap_ranked.png',
    ],
]

titles = [
    ['Arsenic_Prob_5_raw','%_AsExeed5_calculated','%_AsExceed5_ranked',
     'Brackish_ft_raw','Shallow_brackish_calculated', 'Shallow_brackish_ranked',      
    ],
    ['GW_Storage_change_raw','Mag_GW_change_calculated','Abs_GW_change_ranked',
     'PPT_change_raw','Mag_PPT_change_calculated','Abs_PPT_change_ranked',   
    ],
    ['Public_Supply_GW_raw','%_GW_DW_calculated','%_GW_DW_ranked',
     'CoWU2015_raw','IR_WU_calculated','IR_WU_ranked',   
    ],
    ['EPARegulatedSites_raw', 'EPARegulatedSites_calculated','EPARegulatedSites_ranked',
     'Active_Mines_raw', 'Mines_calculated', 'Mines_ranked',   
    ],
    ['No_Ppl_Domestic_supply_raw','Dom_well_pop_calculated','Dom_well_pop_ranked',
     'Demographic_Index_raw','mapSD_calculated','mapSD_ranked',   
    ],
    ['NativePop_raw','Native_pop_den_calculated','Native_pop_den_ranked',
     'Trace_Elements_raw','Data_Gap_calculated', 'Data_Gap_ranked',   
    ],
]

#get height and width of single map...As long as all figures are the same size, this code does not need to change
width = image.size[0]
height = image.size[1]

#create a canvas that fits all 6 maps per figure. (width * maps wanted across, height * maps wanted down)
canvas = Image.new('RGBA',(width * 3, height * 2))

#get maps names and list for x,y coordinates of maps
listx = [0, 1, 2, 0, 1, 2]
listy = [0, 0, 0, 1, 1, 1]
#for saving the figure
name = 1
date = datetime.today().strftime('%Y%m%d')

for mapname, title in zip(mapnames, titles):
    #create a canvas that fits all 6 maps per figure. (width * maps wanted across, height * maps wanted down)
    canvas = Image.new('RGBA',(width * 3, height * 2))
    #open images into canvas
    for m, n, p in zip(mapname, listx, listy):
        image = Image.open(m)
        canvas.paste(image, (width * n, height * p))
    #Labeling the plots
    draw = ImageDraw.Draw(canvas)
    #set up font
    font = ImageFont.truetype(r'../../../../Python/fonts/Roboto/Roboto-Regular.ttf', size=50)
    for n, p, q in zip(listx, listy, title):
        draw.text((width * n + (0.5 *width), height * p), q, font=font, fill = (0,0,0), align = 'center', anchor = 'ma')
    #canvas.show()
    
    # saving the figure
    canvas.save('../'+date+'_6mapvariableset'+str(name)+'.png')
    name = int(name) + 1

# The 12 Map Figure (3 x 4 maps with legend in 5th column) 

In [13]:
#change directory to 'RankedVariables' in a way that maintains original directory
path = cwd + '\RankedVariables' 
os.chdir(path)
#create an object out of my directory
directory = os.scandir(path) 
#loop thorough that object to inspect amount and size of images
n = 1
for file in directory: # for entry in object
    image = Image.open(file.name)
    print(n, image.filename)
    print(image.size)
    n += 1
    
#create list to be used as 'mapnames' variable
directory = os.scandir(path) #this line needs to be repeated...not exactly sure why
for file in directory:
    print("'"+file.name+"',")

1 legend.png
(1056, 816)
2 map%_AsExceed5_ranked.png
(1056, 816)
3 map%_GW_DW_ranked.png
(1056, 816)
4 mapAbs_GW_change_ranked.png
(1056, 816)
5 mapAbs_PPT_change_ranked.png
(1056, 816)
6 mapData_Gap_ranked.png
(1056, 816)
7 mapDom_well_pop_ranked.png
(1056, 816)
8 mapEPARegulatedSites_ranked.png
(1056, 816)
9 mapIR_WU_ranked.png
(1056, 816)
10 mapMines_ranked.png
(1056, 816)
11 mapNative_pop_den_ranked.png
(1056, 816)
12 mapSD_ranked.png
(1056, 816)
13 mapShallow_brackish_ranked.png
(1056, 816)
'legend.png',
'map%_AsExceed5_ranked.png',
'map%_GW_DW_ranked.png',
'mapAbs_GW_change_ranked.png',
'mapAbs_PPT_change_ranked.png',
'mapData_Gap_ranked.png',
'mapDom_well_pop_ranked.png',
'mapEPARegulatedSites_ranked.png',
'mapIR_WU_ranked.png',
'mapMines_ranked.png',
'mapNative_pop_den_ranked.png',
'mapSD_ranked.png',
'mapShallow_brackish_ranked.png',


In [14]:
#get height and width of single map...As long as all figures are the same size, this code does not need to change
width = image.size[0]
height = image.size[1]

#create a canvas that fits all 6 maps per figure. (width * maps wanted across, height * maps wanted down)
canvas = Image.new('RGBA',(width * 3, height * 5)) #height is 5 down to fit the legend

#get maps names and list for x,y coordinates of maps
listx = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
listy = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

#choose order of plots using 'mapnames' (organized in same order as they appear)
mapnames = ['map%_AsExceed5_ranked.png','map%_GW_DW_ranked.png','mapDom_well_pop_ranked.png', 
            'mapShallow_brackish_ranked.png','mapIR_WU_ranked.png','mapSD_ranked.png', 
            'mapAbs_GW_change_ranked.png','mapEPARegulatedSites_ranked.png','mapNative_pop_den_ranked.png',
            'mapAbs_PPT_change_ranked.png','mapMines_ranked.png','mapData_Gap_ranked.png',            
            'legend.png'
           ]
for m, n, p in zip(mapnames, listx, listy):
    image = Image.open(m)
    canvas.paste(image, (width * n, height * p))

In [15]:
#Labeling the plot

#shorten function name
draw = ImageDraw.Draw(canvas)
#set up font
font = ImageFont.truetype(r'../../../../Python/fonts/Roboto/Roboto-Regular.ttf', size=50)
#use previous lists and one more to set up loop (organized in same order as they appear)
titles = ['%_AsExceed5','%_GW_DW','Dom_well_pop',
          'Shallow_brackish','IR_WU','SD',
          'Mag_GW_change','EPA_site_den', 'Native_pop_den',
          'Mag_PPT_change', 'Mines', 'Data_Gap',]                          
for n, p, title in zip(listx, listy, titles):
    draw.text((width * n + (0.5 *width), height * p), title, font=font, fill = (0,0,0), align = 'center', anchor = 'ma')

In [18]:
#Crop the plot
# Setting the points for cropped image
left = 0
top = 0
right = width * 3
bottom = height * 4.5 #removing 1/2 of the legend map block 
 
# Cropped image of above dimension
# (It will not change original image)
canvas_crop = canvas.crop((left, top, right, bottom))
 
# Shows the image in image viewer
canvas_crop.show()

In [17]:
# saving the figure
date = datetime.today().strftime('%Y%m%d')
canvas_crop.save('../'+date+'_12mapfigure.png')

## Extra 1: Interesting aside
code runs with path and obj are given variables...but not if ran in another code

In [60]:
#THIS CODE RUNS!
#inspecting a bunch of images
path = cwd
obj = os.scandir(path) #turn path into an object that is iterable

for entry in obj: #or for file in directory
    image = Image.open(entry.name)
    #image.show()
    print(image.filename)
    print(image.size)
    print(image.format)
    print(image.format_description)

mapCoWU2015_raw.png
(1056, 816)
PNG
Portable network graphics
mapData_Gap_calculated.png
(1056, 816)
PNG
Portable network graphics
mapData_Gap_ranked.png
(1056, 816)
PNG
Portable network graphics
mapIR_WU_calculated.png
(1056, 816)
PNG
Portable network graphics
mapIR_WU_ranked.png
(1056, 816)
PNG
Portable network graphics
mapNEW_TraceALL_Final_Point_raw.png
(1056, 816)
PNG
Portable network graphics


In [51]:
#THIS CODE DOES NOT RUN EVEN THOUGH obj WAS ALREADY DEFINED BEFORE
for entry in obj: #or for file in directory
    image = Image.open(entry.name)
    #image.show()
    print(image.filename)
    print(image.size)
    print(image.format)
    print(image.format_description)

Explanation? Looks like obj needs to be reset. Perhaps when it iterates through, it says at the last iteration (which there are not files after the 6th item)

## Extra 2: New way to figure out what listx and listy should be

In [116]:
# Creating x, y coordinates based on grid size...example (3x2)
##library import
import numpy as np

##take input for how many maps you want across (inputx) and maps down (inputy)
inputx = 3
inputy = 5

##do these commands
iterate_x = list(np.arange(0, inputx, 1)) 
iterate_y = list(np.arange(0, inputy, 1))

#create list for x
listx = []
n = 0
while n < inputy:
    for x in iterate_x:
        listx.append(x)
    n += 1
print(listx)

#create list for y
listy = []
for y in iterate_y:
    n = 0
    while n < inputx:
        listy.append(iterate_y[y])
        n += 1
print(listy)

[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]
